In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from models.resnet20 import build_resnet20  # Ensure this is in your Python path

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# -------------------------
# Dual Gradient-Based Iterative Pruning
# -------------------------
def dual_gradient_iterative_pruning(model, x_train, y_train, sparsity, iterations=10):
    for i in range(iterations):
        with tf.GradientTape() as tape:
            logits = model(x_train[:256], training=True)
            loss = tf.keras.losses.categorical_crossentropy(y_train[:256], logits)
        grads = tape.gradient(loss, model.trainable_weights)

        scores = [tf.abs(g * w) for g, w in zip(grads, model.trainable_weights) if g is not None]
        all_scores = tf.concat([tf.reshape(score, [-1]) for score in scores], axis=0)
        k = int((1 - sparsity) * tf.size(all_scores).numpy())
        threshold = tf.sort(all_scores)[k].numpy()

        masks = [(tf.abs(g * w) > threshold) if g is not None else tf.ones_like(w) for g, w in zip(grads, model.trainable_weights)]

        # Apply masks to weights
        for var, mask in zip(model.trainable_weights, masks):
            var.assign(var * tf.cast(mask, tf.float32))

    print(f"✅ Applied Dual Gradient Iterative Pruning at sparsity {sparsity}")
    return model

# -------------------------
# Training Function
# -------------------------
def train_model(sparsity=0.5, batch_size=128, epochs=120):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    model = build_resnet20()
    model.build(input_shape=(None, 32, 32, 3))
    model.summary()

    # Prune the model
    model = dual_gradient_iterative_pruning(model, x_train, y_train, sparsity)

    # Compile and train
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size)

    # Save model
    model.save(f'models/pruned_resnet20_dual_iterative_sparsity_{sparsity}.h5')
    print("✅ Model training complete and saved.")
    return history

In [2]:
history = train_model(sparsity=0.3, batch_size=128, epochs=150)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [3]:
history = train_model(sparsity=0.5, batch_size=128, epochs=150)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_21 (Conv2D)             (None, 32, 32, 16)   448         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_19 (BatchN  (None, 32, 32, 16)  64          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_19 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_19[0]

In [4]:
history = train_model(sparsity=0.7, batch_size=128, epochs=150)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_42 (Conv2D)             (None, 32, 32, 16)   448         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_38 (BatchN  (None, 32, 32, 16)  64          ['conv2d_42[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_38 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_38[0]

In [5]:
history = train_model(sparsity=0.9, batch_size=128, epochs=150)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_63 (Conv2D)             (None, 32, 32, 16)   448         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_57 (BatchN  (None, 32, 32, 16)  64          ['conv2d_63[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_57 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_57[0]